In [1]:
import numpy as np
import matplotlib.pyplot as plt

# The following few lines of code are unethical and shouldn't be used every again. But they work.
import os
os.environ['PATH'] = "/home/bradlipovsky/anaconda3/envs/fenicsproject/bin:" + os.environ['PATH']

from dolfin import *
from mshr import *

# Physical parameters. These parameters should give 66min fundamental mode period... 
# ... but they dont.
# Units are in GPa-km-g/cm**3-s
E, nu = Constant(80.0), Constant(0.25) # Youngs modulus (GPa) and Poisson Ratio
rho = Constant(5.51) # Average Density, Gkg/km**3 == g/cm**3
R = 6378         # Planet radius, km
mu = E/2./(1+nu) # Lame coefficient for constitutive relation (no need to change)
lmbda = E*nu/(1+nu)/(1-2*nu) #(no need to change)

In [24]:
# Make a mesh. Last parameter is refinement parameter
mesh = generate_mesh( Sphere(Point(R,0,0),R),15 )
coords = mesh.coordinates()

# uncomment for a simple 2D plot
# from pylab import show, triplot
# triplot(coords[:,0], coords[:,1], triangles=mesh.cells())
# show()

# How close is the mesh to a sphere?
np.min(coords[:,0])/R

# Uncomment for fancy 3d plot
# mesh

Generating mesh with CGAL 3D mesh generator


0.0008819900533353147

In [25]:
# Define stress and strain
def eps(v):
    return sym(grad(v))
def sigma(v):
    dim = v.geometric_dimension()
    return 2.0*mu*eps(v) + lmbda*tr(eps(v))*Identity(dim)

# Define function spaces
V = VectorFunctionSpace(mesh, 'Lagrange', degree=1)
u_ = TrialFunction(V)
du = TestFunction(V)

# Boundary conditions
def left(x, on_boundary):
    return x[0] < R * 1e-2

bc = DirichletBC(V, Constant((0.,0.,0.)), left)

# Define variational forms and assemble them
k_form = inner(sigma(du),eps(u_))*dx
l_form = Constant(0.)*u_[0]*dx

K = PETScMatrix()
b = PETScVector()
assemble_system(k_form, l_form, bc, A_tensor=K, b_tensor=b)

m_form = rho*dot(du,u_)*dx
M = PETScMatrix()
assemble(m_form, tensor=M)

In [26]:
# Set up the eigenvalue solver
eigensolver = SLEPcEigenSolver(K, M)
eigensolver.parameters['problem_type'] = 'gen_hermitian'

# Needs to be + or - an order of magnitude of solution in order to ensure stability
eigensolver.parameters['spectral_transform'] = 'shift-and-invert'
eigensolver.parameters['spectral_shift'] = 10000. 

N_eig = 8   # number of eigenvalues
print("Computing {} first eigenvalues...".format(N_eig))
eigensolver.solve(N_eig)

Computing 8 first eigenvalues...


In [27]:
eigenmodes = []
for i in range(N_eig):
    # Extract eigenpair
    r, c, rx, cx = eigensolver.get_eigenpair(i)

    # 3D eigenfrequency
    freq_3D = sqrt(r)/2/pi

    print("Normal mode period: {:8.5f} [s]   ".format(1/freq_3D))

    # Initialize function and assign eigenvector
    eigenmode = Function(V,name="Eigenvector "+str(i))
    eigenmode.vector()[:] = rx
    
    eigenmodes.append(eigenmode)

Normal mode period: 23529.09009 [s]   
Normal mode period: 1189.77624 [s]   
Normal mode period: 1127.07481 [s]   
Normal mode period: 919.81726 [s]   
Normal mode period: 832.43929 [s]   
Normal mode period: 667.09886 [s]   
Normal mode period: 626.41216 [s]   
Normal mode period: 481.43827 [s]   


In [6]:
# We were expecting a fundamental mode around 4000s? What's up with this 23529.09009 s mode?